In [2]:
import pyarrow.parquet as pq
import pandas as pd
import re
from IPython.display import display

#dataset_1 = pq.ParquetDataset('dataset-1/w32processes')
#dataset_2 = pq.ParquetDataset('dataset-2/w32processes')
#dataset_3 = pq.ParquetDataset('dataset-3/w32processes')
dataset_4 = pq.ParquetDataset('../dataset-4/w32processes')

#table_1 = dataset_1.read()
#table_2 = dataset_2.read()
#table_3 = dataset_3.read()
table_4 = dataset_4.read()

#df_1 = table_1.to_pandas()
#df_2 = table_2.to_pandas()
#df_3 = table_3.to_pandas()
df = table_4.to_pandas()

#df_1.to_csv('test_1.csv')
#df_2.to_csv('test_2.csv')
#df_3.to_csv('test_3.csv')
#df.to_csv('test_4.csv')

pd.set_option('display.max_colwidth', None)

print('Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3')

Student ID: 82R4k2JOBFYpjzrPT6xpDrASy2C3


In [3]:
df_elevated = df[df['username'].str.contains('NT AUTHORITY')]

In [4]:
import re
import pandas as pd

# Valores legítimos esperados en Winlogon
LEGIT_SHELL = "explorer.exe"
LEGIT_USERINIT = "userinit.exe"

SUSPICIOUS_DLL_PATTERNS = [
    r"\.dll", r"\.ocx", r"\.scr"  # formatos que podrían usarse como payload
]

def hunt_winlogon_helpers(df):
    """
    Busca posibles abusos de Winlogon Helper DLLs (Notify, Userinit, Shell).
    Basado en procesos y argumentos en df.
    """
    df = df.copy()
    results = []

    # --- 1) winlogon.exe ejecutando con argumentos sospechosos ---
    mask_winlogon = df['name'].str.lower() == "winlogon.exe"
    mask_args_susp = df['arguments'].str.lower().str.contains("|".join(SUSPICIOUS_DLL_PATTERNS), na=False)
    winlogon_susp = df[mask_winlogon & mask_args_susp]
    results.append(winlogon_susp)

    # --- 2) Procesos que deberían ser shell pero no son explorer.exe ---
    mask_shell = df['name'].str.lower().str.contains("shell", na=False) | df['path'].str.lower().str.contains("winlogon", na=False)
    non_explorer = df[(mask_shell) & (~df['name'].str.lower().str.contains(LEGIT_SHELL))]
    results.append(non_explorer)

    # --- 3) Procesos que deberían ser userinit pero no son userinit.exe ---
    mask_userinit = df['name'].str.lower().str.contains("userinit", na=False) | df['path'].str.lower().str.contains("winlogon", na=False)
    non_userinit = df[(mask_userinit) & (~df['name'].str.lower().str.contains(LEGIT_USERINIT))]
    results.append(non_userinit)

    # --- 4) Winlogon o procesos críticos cargando DLLs fuera de System32 ---
    mask_dlls = df['arguments'].str.lower().str.contains("|".join(SUSPICIOUS_DLL_PATTERNS), na=False)
    mask_nonstd_path = ~df['path'].str.lower().str.contains(r'\\windows\\system32\\', na=False)
    dlls_outside_system32 = df[mask_winlogon & mask_dlls & mask_nonstd_path]
    results.append(dlls_outside_system32)

    # --- Combinar resultados ---
    hunt_results = pd.concat(results).drop_duplicates()
    return hunt_results



In [5]:
susp_winlogon = hunt_winlogon_helpers(df)

print("Posibles abusos de Winlogon Helper DLLs:", len(susp_winlogon))
display(susp_winlogon.head(20))


Posibles abusos de Winlogon Helper DLLs: 82


,hostname,name,arguments,username,path,pid
24,EB0005YAT,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,6280
21,EC0002LKS,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,3264
41,EC0002LKS,powershell.exe,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe""",masonwood4,C:\Windows\System32\WindowsPowerShell\v1.0,5390
34,EC0009UFP,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,5856
46,EE0006XXF,powershell.exe,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe""",christiacolli,C:\Windows\System32\WindowsPowerShell\v1.0,3493
51,EL0000LLP,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,4136
11,EL0004LKS,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -noprofile -,lukjacks0,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,1740
23,EL0004YAW,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,4572
69,EL0004YAW,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -noprofile -,lillbarn72,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,3188
49,EL0006ARI,ShellExperienceHost.exe,"""C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy\ShellExperienceHost.exe"" -ServerName:App.AppXtk181tbxbce2qsex02s8tw7hfxa9xb3t.mca",NT AUTHORITY\SYSTEM,C:\Windows\SystemApps\ShellExperienceHost_cw5n1h2txyewy,376
